In [1]:
import sys
sys.path.insert(0, r'P:\pet ML')
import pandas as pd
import numpy as np
import sqlite3
from work_with_db import work_with_db

In [2]:
db = work_with_db()

In [3]:
df = pd.DataFrame(db.action_with_db(action='SELECT', columns=['id', 'Sex', 'Age', 'Height', 'Weight', 'Duration', 'Heart_Rate', 'Body_Temp']))
target = pd.DataFrame(db.action_with_db(action='SELECT', columns=['Calories'], close=True))

In [6]:
df

,0,1,2,3,4,5,6,7
0,0,male,36,189.0,82.0,26.0,101.0,41.0
1,1,female,64,163.0,60.0,8.0,85.0,39.7
2,2,female,51,161.0,64.0,7.0,84.0,39.8
3,3,male,20,192.0,90.0,25.0,105.0,40.7
4,4,female,38,166.0,61.0,25.0,102.0,40.6
...,...,...,...,...,...,...,...,...
999995,999995,female,56,159.0,62.0,6.0,85.0,39.4
999996,999996,male,32,202.0,101.0,3.0,84.0,38.4
999997,999997,female,31,164.0,64.0,14.0,98.0,40.1
999998,999998,female,62,158.0,61.0,25.0,106.0,40.7
